This notebook contains plots produced in the validation sections of the paper

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import seaborn as sns 
sns.set_style('ticks')

import matplotlib.pyplot as plt
import numpy as np
import pymc3 as pm
import scipy.stats as stats

import pickle
import re
import os
import copy

from epimodel import EpidemiologicalParameters, DefaultModel, preprocess_data


from matplotlib.font_manager import FontProperties
fp2 = FontProperties(fname=r"../fonts/Font Awesome 5 Free-Solid-900.otf")

In [ ]:
data = preprocess_data('../merged_data/data_final_nov.csv', last_day='2020-05-30', smoothing=1)
data.mask_reopenings(print_out = False)

In [ ]:
trace = pickle.load(open("traces/final_final_nov.pkl", "rb"))

nS, nCMs = trace.CMReduction.shape
default_res = np.exp(np.log(trace.CMReduction) + np.random.normal(size=(nS, nCMs)) * trace.CMAlphaScales)
default_res_mean = trace.CMReduction

In [ ]:
results_base_dir = "../server_final_nov2/"

def tag_fname_to_label(tag, fname):    
    if tag == "cases_threshold":
        r = re.search(r"(cases_t)(\d*)", fname)
        return r.groups()[1]
    
    if tag == "deaths_threshold":
        r = re.search(r"(deaths_t)(\d*)", fname)
        return r.groups()[1]
    
    if tag == "scaling":
        if "simple" in fname:
            return "Random Constant Scaling"
        elif "variable" in fname:
            return "Time-Varying Correction"
    
    if tag == "npi_leaveout":
        npi_names = data.CMs
        r = re.search(r"(\d*)", fname)[0]
        label = f"{npi_names[int(r[0])]}"
        for npi in r[1:]:
            label += f",\n{npi_names[int(npi)]}"
            
        return label
    
    if tag == "growth_noise":
        npi_names = data.CMs
        r = re.search(r"(growth_noise)-(\d*.\d*)", fname)
        
        return r.groups()[1]
    
    if tag == "iceswe":
        return "Iceland and Sweden Excluded"
            
    if tag == "NPI_prior":
        if "icl" in fname:
            return "Flaxman et. al."
        
        if "half_normal" in fname:
            return "$\\alpha_i \sim$Half Normal$(0.2^2)$"
        
        if "normal" in fname and "0.2" in fname:
            return "$\\alpha_i \sim$Normal$(0, 0.2^2)$"
        
        if "normal" in fname and "10" in fname:
            return "$\\alpha_i \sim$Normal$(0, 10^2)$"
        
    if tag == "oxcgrt":
        npi_names = ['Mask Wearing', 'Travel Screen/Quarantine', 'Travel Bans', 'Public Transport Limited',
                          'Internal Movement Limited', 'Public Information Campaigns', 'Symptomatic Testing']
        r = re.search(r"(\d*)", fname)[0]
        label = f"{npi_names[int(r[0])]}"
        for npi in r[1:]:
            label += f",\n{npi_names[int(npi)]}"
        return label
            
    if tag == "R_prior":
        r = re.search(r"(R_prior-)(\d*.\d*)", fname)
        val = r.groups()[1]
        return val
    
    if tag == "alpha_noise_Scale":
        r = re.search(r"(alpha_noise-)(\d*.\d*)", fname)
        val = r.groups()[1]
        return val

    if tag == "region_holdout":
        return fname[:2]
    
    if tag == "structural":
        if "default" in fname:
            return "Fixed Effects"
        
        if "additive" in fname:
            return "Additive"
        
        if "cases" in fname:
            return "Only Case Data"
        
        if "deaths" in fname:
            return "Only Death Data"
        
        if "noisy" in fname:
            return "Noisy-R (Fixed Effects)"
        
        if "discrete_renewal" in fname:
            return "Discrete Renewal$^*$"
        
    if tag == "alpha_noise_scale":
        an = re.search(r"(alpha_noise-)(\d*.\d*)", fname).groups()[1]
        print(an)
        return an

        
    return f"{tag} - {fname}"

def load_tagged_traces(result_base_dir, tag, extension='-cs.txt'):
    path = os.path.join(results_base_dir, tag)
    
    all_traces = []
    for filedir, _, files in os.walk(path):
        for f in files:
            if extension in f and 'CasesDelay' not in f and 'DeathsDeath' not in f and '_GI_' not in f:
                try:
                    trace = np.loadtxt(os.path.join(filedir, f))
                    all_traces.append([trace, tag_fname_to_label(tag, f)])
                except Exception as e:
                    print(e)
                    
    return all_traces

def search_tagged_traces(result_base_dir, tag, search_key, extension='-cs.txt'):
    path = os.path.join(results_base_dir, tag)
    
    for filedir, _, files in os.walk(path):
        for f in files:
            if extension in f and search_key in f:
                    trace = np.loadtxt(os.path.join(filedir, f))
                    label =  tag_fname_to_label(tag, f)
                    return (trace, label)
    
    print('No experiment found')
    return None

In [ ]:
colors = [*sns.color_palette("colorblind"), *sns.color_palette("dark")]

derived_features_comb = [
 ("Gatherings limited to 1000\npeople or less", [0]),
 ("Gatherings limited to 100\npeople or less", [0, 1]),
 ("Gatherings limited to 10\npeople or less", [0, 1, 2]),
 ("Some businesses closed", [3]),
 ("Most businesses closed", [3, 4]),
 ("Schools and universities\nclosed", [5, 6]),
 ("Stay-at-home order\n(Additional Effect)", [7]),
 ]


derived_features_uncomb = [ #()"Healthcare Infection Control", [0]),
 ("Gatherings limited to 1000\npeople or less", [0]),
 ("Gatherings limited to 100\npeople or less", [1]),
 ("Gatherings limited to 10\npeople or less", [2]),
 ("Some businesses closed", [3]),
 ("Most businesses closed", [4]),
 ("Schools and universities\nclosed", [5, 6]),
 ("Stay-at-home order\n(with exemptions)", [7]),
 ]

derived_features = derived_features_comb

cols = sns.cubehelix_palette(3, start=0.2, light=0.6, dark=0.1, rot=0.2)
cm_plot_style = [
            ("\uf0c0", (0.922, 0.564, 0.612)), # ppl
            ("\uf0c0", (0.671, 0.290, 0.341)), # ppl
            ("\uf0c0", (0.211, 0.086, 0.011)), # ppl
            ("\uf07a", (0.316, 0.506, 0.420)), # shop 1
            ("\uf07a", (0.082, 0.196, 0.110)), # shop2
            ("\uf549", (0.098, 0.364, 0.58)), # school
            ("\uf19d", (0.0, 0.22, 0.4)), # university
            ("\uf965", (0.803, 0.376, 0)) # home
]

def produce_ranges(trace):
    means = np.mean(trace, axis=0)
    med = np.median(trace, axis=0)
    li = np.percentile(trace, 2.5, axis=0)
    ui = np.percentile(trace, 97.5, axis=0)
    lq = np.percentile(trace, 25, axis=0)
    uq = np.percentile(trace, 75, axis=0)
    return means, med, li, ui, lq, uq

def add_trace_to_plot(res, y_off, col, label, alpha, width, size=8, zeros=None):
    nS, _ = res.shape
    nF = len(derived_features)
    derived_samples = np.zeros((nS, nF))

    for f_i, (f, prodrows) in enumerate(derived_features):
        samples = np.ones(nS)
        for r in prodrows:
            samples = samples * res[:, r] 
        derived_samples[:, f_i] = samples

    res = derived_samples
    res = 100*(1-res)
    
    if zeros is not None:
        for z in zeros:
            if z < nF:
                res[:, z] = 1e5
    
    y_vals = -1 * np.arange(nF)
    plt.plot([100], [100], color=col, linewidth=1, alpha=alpha, label=label)
    mn, med, li, ui, lq, uq = produce_ranges(res)
    plt.scatter(med, y_vals+y_off, marker="|", color=col, s=size, alpha=alpha)
    for cm in range(nF):
        plt.plot([li[cm], ui[cm]], [y_vals[cm]+y_off, y_vals[cm]+y_off], color=col, alpha=alpha*0.25, linewidth=width)
        plt.plot([lq[cm], uq[cm]], [y_vals[cm]+y_off, y_vals[cm]+y_off], color=col, alpha=alpha*0.75, linewidth=width)
        
def add_additive_trace_to_plot(res, y_off, col, label, alpha, width, size=8, zeros=None):
    nS, _ = res.shape
    nF = len(derived_features)
    derived_samples = np.zeros((nS, nF))

    for f_i, (f, prodrows) in enumerate(derived_features):
        samples = np.zeros(nS)
        for r in prodrows:
            samples = samples + res[:, r] 
        derived_samples[:, f_i] = samples

    res = derived_samples
    res = 100*res
    
    if zeros is not None:
        for z in zeros:
            if z < nF:
                res[:, z] = 1e5
                
    print(np.median(derived_samples, axis=0))
    
    y_vals = -1 * np.arange(nF)
    plt.plot([100], [100], color=col, linewidth=1, alpha=alpha, label=label)
    mn, med, li, ui, lq, uq = produce_ranges(res)
    plt.scatter(med, y_vals+y_off, marker="|", color=col, s=size, alpha=alpha)
    for cm in range(nF):
        if cm > 7:
            plt.plot([li[cm], ui[cm]], [y_vals[cm]+y_off, y_vals[cm]+y_off], color=col, alpha=alpha*0.25*0.25, linewidth=width, linestyle="--")
            plt.plot([lq[cm], uq[cm]], [y_vals[cm]+y_off, y_vals[cm]+y_off], color=col, alpha=alpha*0.75*0.25, linewidth=width, linestyle="--")
        else:
            plt.plot([li[cm], ui[cm]], [y_vals[cm]+y_off, y_vals[cm]+y_off], color=col, alpha=alpha*0.25, linewidth=width)
            plt.plot([lq[cm], uq[cm]], [y_vals[cm]+y_off, y_vals[cm]+y_off], color=col, alpha=alpha*0.75, linewidth=width)
        
        
        
def setup_large_plot(y_ticks = True, icons = True, xlabel=True):
    nF = len(derived_features)
    ax = plt.gca()
    x_min = -25
    x_max = 75
    plt.plot([0, 0], [1, -(nF+2)], "--k", linewidth=0.5)
    
#     plt.plot([17.5, 17.5], [1, -(nF+2)], "--", linewidth=0.5, color=cols[0])
#     plt.plot([35, 35], [1, -(nF+2)], "--", linewidth=0.5, color=cols[0])
    
    xrange = np.array([x_min, x_max])
    for height in range(0, nF+2, 2):
        plt.fill_between(xrange, -(height-0.5), -(height+0.5), color="silver", alpha=0.5, linewidth=0)
    xtick_vals = [-25, 0, 25, 50, 75, 100]
    xtick_str = [f"{x:.0f}%" for x in xtick_vals]
    if y_ticks:
        plt.yticks(-np.arange(nF), [f"{f[0]}" for f in derived_features], fontsize=8, ha="left")
        yax = ax.get_yaxis()
        yax.set_tick_params(pad=125)
        
    else:
        plt.yticks([])
    x_r = np.abs(x_min - x_max)
    plt.xticks(xtick_vals, xtick_str, fontsize=8)
    plt.xlim([x_min, x_max])
    plt.ylim([-(nF - 0.25), 0.75])
    
    if icons:
        for cm in range(len(derived_features)):
            for i, val in enumerate(derived_features[cm][1]):
                plt.text(x_min - 7.5 - 8*i, -cm, cm_plot_style[val][0], horizontalalignment='center', verticalalignment='center',
                             fontproperties=fp2, fontsize=8, color=cm_plot_style[val][1], zorder=-i, alpha = 1) 
                
    if xlabel:
        plt.xlabel("Reduction in $R_t$\nin the context of our data", fontsize=8)
        
def setup_larger_plot(y_ticks = True, icons = True, xlabel=True):
    nF = len(derived_features)
    ax = plt.gca()
    x_min = -25
    x_max = 75
    plt.plot([0, 0], [1, -(nF+2)], "--k", linewidth=0.5)
#     plt.plot([17.5, 17.5], [1, -(nF+2)], "--", linewidth=0.5, color=cols[0])
#     plt.plot([35, 35], [1, -(nF+2)], "--", linewidth=0.5, color=cols[0])
    xrange = np.array([x_min, x_max])
    for height in range(0, nF+2, 2):
        plt.fill_between(xrange, -(height-0.5), -(height+0.5), color="silver", alpha=0.5, linewidth=0)
    xtick_vals = [-25, 0, 25, 50, 75, 100]
    xtick_str = [f"{x:.0f}%" for x in xtick_vals]
    if y_ticks:
        plt.yticks(-np.arange(nF), [f"{f[0]}" for f in derived_features], fontsize=8, ha="left")
        yax = ax.get_yaxis()
        yax.set_tick_params(pad=125)
        
    else:
        plt.yticks([])
    x_r = np.abs(x_min - x_max)
    plt.xticks(xtick_vals, xtick_str, fontsize=8)
    plt.xlim([x_min, x_max])
    plt.ylim([-(nF - 0.25), 0.75])
    
    if icons:
        for cm in range(len(derived_features)):
            for i, val in enumerate(derived_features[cm][1]):
                plt.text(x_min - 7.5 - 9*i, -cm, cm_plot_style[val][0], horizontalalignment='center', verticalalignment='center',
                             fontproperties=fp2, fontsize=12, color=cm_plot_style[val][1], zorder=-i, alpha = 1) 
                
    if xlabel:
        plt.xlabel("Reduction in $R_t$\nin the context of our data", fontsize=8)
    
def setup_small_plot(y_ticks = True, icons = True, xlabel=True):
    nF = len(derived_features)
    ax = plt.gca()
    x_min = -25
    x_max = 75
    plt.plot([0, 0], [1, -(nF+2)], "--k", linewidth=0.5)
#     plt.plot([17.5, 17.5], [1, -(nF+2)], "--", linewidth=0.5, color=cols[0])
#     plt.plot([35, 35], [1, -(nF+2)], "--", linewidth=0.5, color=cols[0])
    xrange = np.array([x_min, x_max])
    for height in range(0, nF+2, 2):
        plt.fill_between(xrange, -(height-0.5), -(height+0.5), color="silver", alpha=0.5, linewidth=0)
    xtick_vals = [0, 50, 100]
    xtick_str = [f"{x:.0f}%" for x in xtick_vals]
    if y_ticks:
        plt.yticks(-np.arange(nF), [f"{f[0]}" for f in derived_features], fontsize=8, ha="left")
        yax = ax.get_yaxis()
        yax.set_tick_params(pad=125)
        
    else:
        plt.yticks([])
    x_r = np.abs(x_min - x_max)
    plt.xticks(xtick_vals, xtick_str, fontsize=8)
    plt.xlim([x_min, x_max])
    plt.ylim([-(nF - 0.25), 0.75])
    
    if icons:
        for cm in range(len(derived_features)):
            for i, val in enumerate(derived_features[cm][1]):
                plt.text(x_min - 7.5 - 13*i, -cm, cm_plot_style[val][0], horizontalalignment='center', verticalalignment='center',
                             fontproperties=fp2, fontsize=8, color=cm_plot_style[val][1], zorder=-i, alpha = 1) 
                
    if xlabel:
        plt.xlabel("Reduction in $R_t$\nin the context of our data", fontsize=8)
        
class ResultsObject():
    def __init__(self, indx, trace):
        self.CMReduction = trace.CMReduction
        self.RegionR = trace.RegionR[:, indx]
        self.InfectedCases = trace.InfectedCases[:, indx, :]
        self.InfectedDeaths = trace.InfectedDeaths[:, indx, :]
        self.ExpectedCases = trace.ExpectedCases[:, indx, :]
        self.ExpectedDeaths = trace.ExpectedDeaths[:, indx, :]

In [ ]:
def combine_trace(trace, derived_features):
    nS, _ = trace.shape
    nCMs = len(derived_features)
    derived_samples = np.zeros((nS, nCMs))

    for f_i, (f, prodrows) in enumerate(derived_features):
        samples = np.ones(nS)
        for r in prodrows:
            samples = samples * trace[:, r] 
        derived_samples[:, f_i] = samples

    res = copy.deepcopy(derived_samples)
    res = 100*(1-res)
    
    return res

In [ ]:
def produce_score(default_res, traces, exclude=None):
    default_med = 1 - np.median(default_res, axis=0)
    
    regression_table = np.zeros((0, 2))
    labels_used = []
    
    for trace, label in traces:
        if exclude is not None:
            if isinstance(exclude, list):
                flags = [e in label for e in exclude]
                if any(flags):
                    continue
            else:
                if exclude in label:
                    continue
        
        med = 1 - np.median(trace[:, :9], axis=0)
        med = 1 - med if 'Additive' in label else med
        
        regression_table = np.append(regression_table, np.vstack([default_med, med]).T, axis=0)
        labels_used.append(label)
    
    mask = ~ np.any(np.isnan(regression_table), axis=-1)
    _, _, score, _, _ = stats.linregress(regression_table[mask])
    
    print(f'Produced stored using {[labels_used]}')
    return score

In [ ]:
def produce_score_sd(default_res, traces, derived_features, exclude=None):
    default_med = np.median(combine_trace(default_res, derived_features), axis=0)
    
    median_values = np.zeros((0, len(default_med)))
    median_values = np.append(median_values, default_med.reshape((1, len(derived_features))), axis=0)
    labels_used = []
    
    
    for trace, label in traces:
        if exclude is not None:
            if isinstance(exclude, list):
                flags = [e in label for e in exclude]
                if any(flags):
                    continue
            else:
                if exclude in label:
                    continue
        
        med = np.median(combine_trace(trace[:, :8], derived_features), axis=0)
        median_values = np.append(median_values, med.reshape((1, len(derived_features))), axis=0)
        
    med_stds = np.nanstd(median_values, axis=0)
    print(med_stds)
    score = np.mean(med_stds)
    print(score)
    
    print(f'Produced stored using {[labels_used]}')
    return score

# Category 1: Structural Sensitivity

In [ ]:
traces = load_tagged_traces(results_base_dir, 'structural', '.txt')
score = produce_score_sd(default_res, traces, derived_features_comb, ['Only', 'Unknown', 'Additive'])
traces = [(trace, label) for trace, label in traces if ('Only' not in label and 'Additive' not in label and 'Unknown' not in label)]

fig = plt.figure(figsize=(8, 3), dpi=300)
plt.subplot(121)
setup_large_plot()
y_off = -np.linspace(-0.2, 0.2, len(traces)+1)
width = 1
for i, (trace, label) in enumerate(traces):
    add_trace_to_plot(trace, y_off[i], colors[i], label, 1, width) 
add_trace_to_plot(default_res_mean, y_off[-1], 'k', "Default", 1, width) 
plt.xlabel("Average reduction in $R_t$,\nin the context of our data", fontsize=8)

plt.legend(shadow=True, fancybox=True, loc="upper left", bbox_to_anchor=(0.85, 1.01), fontsize=6, title=f'$\\bar{{\\sigma}}={score:.2f}\\%$', title_fontsize=8)
plt.title("Structural Sensitivity: Multiplicative Models", fontsize=10)

plt.subplot(122)
setup_large_plot(False, True, True)
trace, label = search_tagged_traces(results_base_dir, 'structural', 'additive', '.txt')
add_additive_trace_to_plot(trace, 0, colors[0], label, 1, width)
plt.title("Structural Sensitivity: Additive Model", fontsize=10)
plt.xlabel("Average reduction in $R_t$,\nas a percentage of $R_0$,\nin the context of our data", fontsize=8)

plt.tight_layout()
plt.savefig(f"figs/validation/FigureSSA.pdf", bbox_inches='tight')

# Category 2: Unobserved Effects

In [ ]:
fig = plt.figure(figsize=(8.5, 3), dpi=300)

plt.subplot(122)
setup_large_plot(False, True, True)

traces = load_tagged_traces(results_base_dir, "oxcgrt")
testing_index = [n for _, n in traces].index('Symptomatic Testing')
mask_index = [n for _, n in traces].index('Mask Wearing')
orig = copy.deepcopy(traces[testing_index][0])
traces[testing_index][0] = orig[:, [1, 2, 3, 4, 5, 6, 7, 8]]
orig = copy.deepcopy(traces[mask_index][0])
traces[mask_index][0] = orig[:, [1, 2, 3, 4, 5, 6, 7, 8]]

# score = produce_score(default_res, traces)
score = produce_score_sd(default_res, traces, derived_features_comb)

y_off = -np.linspace(-0.3, 0.3, len(traces)+1)
width = 1
for i, (trace, label) in enumerate(traces):
    add_trace_to_plot(trace[:, :9], y_off[i], colors[i], label, 1, width)
add_trace_to_plot(default_res, y_off[-1], 'k', "Default", 1, width) 

score = produce_score_sd(default_res, traces, derived_features_comb)
plt.legend(shadow=True, fancybox=True, loc="upper left", bbox_to_anchor=(0.8, 1.01), fontsize=6, title=f'$\\bar{{\\sigma}}={score:.2f}\\%$', title_fontsize=8)
plt.title("Including Additional NPIs", fontsize="10")

plt.subplot(121)
derived_features = derived_features_uncomb
setup_large_plot(True, True, True)
traces = load_tagged_traces(results_base_dir, "npi_leaveout")

# remove values which are invalid!
labels = [l for _, l in traces]
for l_i, label in enumerate(labels):
    for cm_i, cm in enumerate(data.CMs):
        if '<' not in cm and cm in label:
            traces[l_i][0][:, cm_i] = np.nan
        if cm == label:
            traces[l_i][0][:, cm_i] = np.nan
        
score = produce_score_sd(default_res, traces, derived_features_uncomb)

order = [*data.CMs]
t_is = [labels.index(o) for o in order]

y_off = -np.linspace(-0.3, 0.3, len(traces)+1)
width = 1
for i, t_i in enumerate(t_is):
    trace, label = traces[t_i]
    add_trace_to_plot(trace, y_off[i], colors[i], label, 1, width)
add_trace_to_plot(default_res, y_off[-1], 'k', "Default", 1, width) 
    
derived_features = derived_features_comb

plt.legend(shadow=True, fancybox=True, loc="upper left", bbox_to_anchor=(0.8, 1.01), fontsize=6, title=f'$\\bar{{\\sigma}}={score:.2f}\\%$', title_fontsize=8)
plt.title("Excluding Collected NPIs\nUncombined Effects Shown", fontsize="10")
plt.tight_layout(w_pad=0)
plt.savefig(f"figs/validation/FigureUOB.pdf", bbox_inches='tight')

In [ ]:
labels

# data sensitivity

#### Data Type

In [ ]:
fig = plt.figure(figsize=(6.75, 3), dpi=300)
setup_large_plot()
traces = [search_tagged_traces(results_base_dir, 'structural', 'cases'), search_tagged_traces(results_base_dir, 'structural', 'deaths')]
score = produce_score(default_res, traces)
y_off = -np.linspace(-0.2, 0.2, len(traces)+1)
colors_h = [colors[0], colors[3]]
width = 2
for i, (trace, label) in enumerate(traces):
    add_trace_to_plot(trace, y_off[i], colors_h[i], label, 1, width)
add_trace_to_plot(default_res, y_off[-1], 'k', "Default", 1, width)  

score = produce_score_sd(default_res, traces, derived_features_comb)
plt.legend(shadow=True, fancybox=True, loc="upper right", bbox_to_anchor=(1.55, 1.01), fontsize=6, title=f'$\\bar{{\\sigma}}={score:.2f}\\%$', title_fontsize=8)
plt.title("Data Type", fontsize=10)

plt.tight_layout()
plt.savefig(f"figs/validation/FigureDataType.pdf", bbox_inches='tight')

#### Country Sensitivity

In [ ]:
fig = plt.figure(figsize=(10.5, 5.5), dpi=300)

traces = load_tagged_traces(results_base_dir, 'region_holdout')
score = produce_score_sd(default_res, traces, derived_features_comb)

y_off = -np.linspace(-0.4, 0.4, 8)
for r_i, r in enumerate(data.Rs):
    r_i_p = r_i % 7 
    if r_i_p == 0:
        plt.legend(shadow=True, fancybox=True, loc="upper left", bbox_to_anchor=(0.8, 1.01), fontsize=6, title=f'$\\bar{{\\sigma}}={score:.2f}\\%$', title_fontsize=8)
        plt.subplot(2, 3, int(r_i/7) + 1)
        
        xlabel = False if (int(r_i/7) + 1) < 4 else True
        
        if (int(r_i/7) + 1) == 1 or (int(r_i/7) + 1) == 4:
            setup_large_plot(xlabel=xlabel)
        else:
            setup_large_plot(False, xlabel=xlabel)
        add_trace_to_plot(default_res, y_off[-1], "k", "Default", 1, 1)
    trace, _ = search_tagged_traces(results_base_dir, 'region_holdout', r)
    label = data.RNames[r][0]
    if 'and' in label and 'Bosnia' in label:
        label = 'Bosnia and\nHerzegovina'
    add_trace_to_plot(trace, y_off[r_i_p], colors[r_i_p], label, 1, 1)
    
#     plt.legend(shadow=True, fancybox=True, loc="upper right", bbox_to_anchor=(0.8, 1.01), fontsize=8, title=f'$\\bar{{\\sigma}}={score:.2f}\\%$', title_fontsize=10)
plt.legend(shadow=True, fancybox=True, loc="upper left", bbox_to_anchor=(0.8, 1.01), fontsize=6, title=f'$\\bar{{\\sigma}}={score:.2f}\\%$', title_fontsize=8)
plt.tight_layout(w_pad=0.15)
plt.savefig(f"figs/validation/FigureCountryLeaveout.pdf", bbox_inches='tight')

#### Thresholds

In [ ]:
fig = plt.figure(figsize=(8, 3), dpi=300)

plt.subplot(121)
setup_large_plot()

traces = load_tagged_traces(results_base_dir, "cases_threshold")
traces = [(trace, label) for trace, label in traces if '10' not in label]
score = produce_score_sd(default_res, traces, derived_features_comb)

y_off = -np.linspace(-0.25, 0.25, len(traces)+1)
width = 1
traces.sort(key= lambda x: int(x[1]))
for i, (trace, label) in enumerate(traces):
    if i<2:
        add_trace_to_plot(trace[:, :9], y_off[i], colors[i], label, 1, width)
        
    if i == 2:
        add_trace_to_plot(default_res, y_off[i], 'k', "Default (100)", 1, width)
    
    if i > 1:
        add_trace_to_plot(trace[:, :9], y_off[i+1], colors[i], label, 1, width)

plt.legend(shadow=True, fancybox=True, loc="upper left", bbox_to_anchor=(0.95, 1.01), fontsize=6, title=f'$\\bar{{\\sigma}}={score:.2f}\\%$', title_fontsize=8)
plt.title("Minimum Cases Threshold", fontsize="10")

plt.subplot(122)
setup_large_plot(False, True, True)
traces = load_tagged_traces(results_base_dir, "deaths_threshold")
score = produce_score_sd(default_res, traces, derived_features_comb)

traces.sort(key= lambda x: int(x[1]))

y_off = -np.linspace(-0.25, 0.25, len(traces)+1)
width = 1
for i, (trace, label) in enumerate(traces):
    if i<2:
        add_trace_to_plot(trace[:, :9], y_off[i], colors[i], label, 1, width)
        
    if i == 2:
        add_trace_to_plot(default_res, y_off[i], 'k', "Default (10)", 1, width)
    
    if i > 1:
        add_trace_to_plot(trace[:, :9], y_off[i+1], colors[i], label, 1, width)
 
plt.legend(shadow=True, fancybox=True, loc="upper left", bbox_to_anchor=(0.95, 1.01), fontsize=6, title=f'$\\bar{{\\sigma}}={score:.2f}\\%$', title_fontsize=8)
plt.title("Minimum Deaths Threshold", fontsize="10")
plt.tight_layout()
plt.savefig(f"figs/validation/FigureThresholds.pdf", bbox_inches='tight')

# additional epi

In [ ]:
fig = plt.figure(figsize=(8, 3), dpi=300)

plt.subplot(121)
setup_large_plot()

traces = load_tagged_traces(results_base_dir, "R_prior")
score = produce_score_sd(default_res, traces, derived_features_comb)

y_off = -np.linspace(-0.25, 0.25, len(traces)+1)
width = 1
traces.sort(key= lambda x: float(x[1]))
for i, (trace, label) in enumerate(traces):
    if i<2:
        add_trace_to_plot(trace[:, :9], y_off[i], colors[i], f'$\\mu[R] ={label}$', 1, width)
        
    if i == 2:
        add_trace_to_plot(default_res, y_off[i], 'k', f'Default\n$\\mu[R] =3.28$', 1, width)
    
    if i > 1:
        add_trace_to_plot(trace[:, :9], y_off[i+1], colors[i], f'$\\mu[R] ={label}$', 1, width)

plt.legend(shadow=True, fancybox=True, loc="upper left", bbox_to_anchor=(0.95, 1.01), fontsize=6, title=f'$\\bar{{\\sigma}}={score:.2f}\\%$', title_fontsize=8)
plt.title("$R_0$ prior", fontsize=10)

plt.subplot(122)
setup_large_plot(False, True, True)
traces = load_tagged_traces(results_base_dir, "NPI_prior")
score = produce_score_sd(default_res, traces, derived_features_comb)

# traces.sort(key= lambda x: int(x[1]))

y_off = -np.linspace(-0.25, 0.25, len(traces)+1)
width = 1
for i, (trace, label) in enumerate(traces):
    add_trace_to_plot(trace[:, :9], y_off[i], colors[i], label, 1, width)
add_trace_to_plot(default_res, y_off[-1], 'k', "Default", 1, width) 
    
plt.legend(shadow=True, fancybox=True, loc="upper left", bbox_to_anchor=(0.95, 1.01), fontsize=6, title=f'$\\bar{{\\sigma}}={score:.2f}\\%$', title_fontsize=8)
plt.title("NPI Effectiveness Prior", fontsize=10)
plt.tight_layout()
plt.savefig(f"figs/validation/FigureAdditionalEpi.pdf", bbox_inches='tight')

# Iceland, Sweden Plot

In [ ]:
plt.figure(figsize=(4,3), dpi=300)

derived_features = [
 ("Schools closed", [5]),
 ("Universities closed", [6]),
 ("Schools and univerisities closed", [5, 6]),
 ]

nF = len(derived_features)

ax = plt.gca()
x_min = -25
x_max = 100
plt.plot([0, 0], [1, -(nF)], "--k", linewidth=0.5)
xrange = np.array([x_min, x_max])
for height in range(0, nF+2, 2):
    plt.fill_between(xrange, -(height-0.5), -(height+0.5), color="silver", alpha=0.5, linewidth=0)
xtick_vals = [-25, 0, 25, 50, 75, 100]
xtick_str = [f"{x:.0f}%" for x in xtick_vals]

plt.yticks(-np.arange(nF), [f"{f[0]}" for f in derived_features], fontsize=8, ha="left")
yax = ax.get_yaxis()
yax.set_tick_params(pad=150)

x_r = np.abs(x_min - x_max)
plt.xticks(xtick_vals, xtick_str, fontsize=8)
plt.xlim([x_min, x_max])
plt.ylim([-(nF - 0.25), 0.75])

for cm in range(len(derived_features)):
    for i, val in enumerate(derived_features[cm][1]):
        plt.text(x_min - 7.5 - 8*i, -cm, cm_plot_style[val][0], horizontalalignment='center', verticalalignment='center',
                     fontproperties=fp2, fontsize=8, color=cm_plot_style[val][1], zorder=-i, alpha = 1) 

plt.xlabel("Average reduction in $R_t$,\nin the context of our data", fontsize=8)


iceland = search_tagged_traces(results_base_dir, 'region_holdout', 'IS')[0]
sweden = search_tagged_traces(results_base_dir, 'region_holdout', 'SE')[0]
iceswe = search_tagged_traces(results_base_dir, 'iceswe', 'iceswe')[0]
y_off = -np.linspace(-0.2, 0.2, 4)
add_trace_to_plot(iceland, y_off[0], colors[0], "Iceland Excluded", 1, 2)
add_trace_to_plot(sweden, y_off[1], colors[1], "Sweden Excluded", 1, 2)
add_trace_to_plot(iceswe, y_off[2], colors[2], "Iceland and\nSweden Excluded", 1, 2)
add_trace_to_plot(default_res, y_off[-1], "k", "Default", 1, 2)
plt.legend(shadow=True, fancybox=True, fontsize=6)
plt.title("Schools and Universities Sensitivity", fontsize=10)
plt.savefig(f"figs/validation/FigureISSE.pdf", bbox_inches='tight')

# Global Sensitivity

In [ ]:
derived_features = derived_features_comb

In [ ]:
def fname_to_params(fname):
    gi = float(re.search(r"(gi_mean_mean)-(\d*.\d*)", fname).groups()[1])
    cd = float(re.search(r"(cases_mean_mean)-(\d*.\d*)", fname).groups()[1])
    dd = float(re.search(r"(deaths_mean_mean)-(\d*.\d*)", fname).groups()[1])
    
    return gi, cd, dd

def load_tagged_traces_gs(result_base_dir, tag):
    path = os.path.join(results_base_dir, tag)
    
    all_traces = []
    for filedir, _, files in os.walk(path):
        for f in files:
            if '-cs.txt' in f:
                try:
                    trace = np.loadtxt(os.path.join(filedir, f))
                    
                    nS, _ = trace.shape
                    nF = len(derived_features)
                    derived_samples = np.zeros((nS, nF))

                    for f_i, (_, prodrows) in enumerate(derived_features):
                        samples = np.ones(nS)
                        for r in prodrows:
                            samples = samples * trace[:, r] 
                        derived_samples[:, f_i] = samples

                    res = derived_samples
                    median = np.median(res, axis=0)
                    
                    gi, cd, dd = fname_to_params(f)
                    
                    all_traces.append([median, gi, cd, dd, trace])
                    print(f'{len(all_traces)} Exps')
                except Exception as e:
                    print(e)
                    
    return all_traces

In [ ]:
global_traces = load_tagged_traces_gs(results_base_dir, 'epiparam_prior')

In [ ]:
medians, gis, cds, dds, full_traces = list(zip(*global_traces))

In [ ]:
produce_score_sd(default_res, [(t, '') for t in full_traces], derived_features_comb)

In [ ]:
nS, _ = default_res.shape
nF = len(derived_features)
derived_samples = np.zeros((nS, nF))

for f_i, (_, prodrows) in enumerate(derived_features):
    samples = np.ones(nS)
    for r in prodrows:
        samples = samples * default_res[:, r] 
    derived_samples[:, f_i] = samples
    
default_median = np.median(derived_samples, axis=0)

In [ ]:
unique_gis = np.unique(gis)
unique_cds = np.unique(cds)
unique_dds = np.unique(dds)

In [ ]:
def gi_to_grid(cm, global_traces, unique_cds, unique_dds, gi, default_median):
    filtered_traces = list(filter(lambda t: t[1] == gi, global_traces))
    
    grid = np.zeros((len(unique_cds), len(unique_dds)))
    grid[:] = np.nan
    
    for meds, _, cd, dd, _ in filtered_traces:
        cd_ind = np.where(unique_cds == cd)
        dd_ind = np.where(unique_dds == dd)
        val = 100*(-meds[cm] + default_median[cm]) 
        grid[cd_ind, dd_ind] = val
    
    return grid

In [ ]:
short_names = ['Gatherings <1000', 'Gatherings <100', 'Gatherings <10', 'Some Buss', 'Most Buss', 'Educat', 'Stay Home']

In [ ]:
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import PercentFormatter

current_cmap = mpl.cm.get_cmap('PuOr_r')
current_cmap.set_bad(color='k')

plt.figure(figsize=(10, 12), dpi=300)
for cm in range(7):
    for gi_ind, gi in enumerate(unique_gis.tolist()):
        plt.subplot(7, 5, 5*cm + gi_ind + 1)
        grid = gi_to_grid(cm, global_traces, unique_cds, unique_dds, gi, default_median)
        im = plt.imshow(grid, cmap=current_cmap, origin='lower', vmin=-15, vmax=15)
        plt.xticks([])
        plt.yticks([])
        
        if cm == 0:
            plt.title(f'$\\mu_{{GI}}={gi:.2f}$', fontsize=10)
        
        if gi_ind == 0:
            plt.ylabel(f'$\\bf{short_names[cm]}$\n\n Infection to Case\nConfirmation Delay (Days)', fontsize=8)
            plt.yticks(np.arange(5), unique_cds, fontsize=6)
            
        if cm == 6:
            plt.xlabel(f'Infection to Death Delay (Days)', fontsize=8)
            plt.xticks(np.arange(5), unique_dds, fontsize=6)
            
        ax = plt.gca()
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="5%", pad=0.05)
        cbr = plt.colorbar(im, cax=cax, format=PercentFormatter())
        ax = plt.gca()
        ax.tick_params(axis="both", which="major", labelsize=7)
        cbr.set_ticks([-15, -7.5, 0, 7.5, 15])
#         cbr.set_label('$\\Delta$(Median) Effectivenss\n Estimate', fontsize=8)
        
plt.tight_layout(w_pad=0.75, h_pad=-8)
plt.savefig(f"figs/validation/FigureGlobalSAAll.pdf", bbox_inches='tight')

### Compressed Version

In [ ]:
def combine_traces(filter_lambda, global_traces):
    filtered_traces = list(filter(filter_lambda, global_traces))
    
    nF = 8
    samples = np.zeros((0, nF))
    
    for _, _, _, _, t in filtered_traces:
        samples = np.append(samples, t, axis=0)
    
    return samples

In [ ]:
cd_traces = []
for unique_cd in unique_cds:
    cd_traces.append((combine_traces(lambda t: t[2] == unique_cd, global_traces), f'Mean {unique_cd} days'))
    
gi_traces = []
for unique_gi in unique_gis:
    gi_traces.append((combine_traces(lambda t: t[1] == unique_gi, global_traces), f'Mean {unique_gi} days'))
    
dd_traces = []
for unique_dd in unique_dds:
    dd_traces.append((combine_traces(lambda t: t[3] == unique_dd, global_traces), f'Mean {unique_dd} days'))

In [ ]:
fig = plt.figure(figsize=(10.25, 3), dpi=300)
plt.subplot(131)
setup_large_plot()
score = produce_score_sd(default_res, cd_traces, derived_features_comb)
y_off = -np.linspace(-0.3, 0.3, len(cd_traces))
width = 1
for i, (trace, label) in enumerate(cd_traces):
    add_trace_to_plot(trace, y_off[i], colors[i], label, 1, width)

plt.legend(shadow=True, fancybox=True, loc="upper right", bbox_to_anchor=(1.35, 1.01), fontsize=6, title=f'$\\bar{{\\sigma}}={score:.2f}\\%$', title_fontsize=8)
plt.title("Infection to Case-Confirmation Delay", fontsize="10")

plt.subplot(132)
score = produce_score_sd(default_res, dd_traces, derived_features_comb)
setup_large_plot(False, True, True)
y_off = -np.linspace(-0.3, 0.3, len(dd_traces))
width = 1
for i, (trace, label) in enumerate(dd_traces):
    add_trace_to_plot(trace[:, :9], y_off[i], colors[i], label, 1, width)

plt.legend(shadow=True, fancybox=True, loc="upper right", bbox_to_anchor=(1.35, 1.01), fontsize=6, title=f'$\\bar{{\\sigma}}={score:.2f}\\%$', title_fontsize=8)
plt.title("Infection to Death Delay", fontsize="10")

plt.subplot(133)
score = produce_score_sd(default_res, gi_traces, derived_features_comb)
setup_large_plot(False, True, True)
y_off = -np.linspace(-0.3, 0.3, len(gi_traces))
width = 1
for i, (trace, label) in enumerate(gi_traces):
    add_trace_to_plot(trace[:, :9], y_off[i], colors[i], label, 1, width)

plt.legend(shadow=True, fancybox=True, loc="upper right", bbox_to_anchor=(1.35, 1.01), fontsize=6, title=f'$\\bar{{\\sigma}}={score:.2f}\\%$', title_fontsize=8)
plt.title("Generation Interval", fontsize="10")

plt.tight_layout(w_pad=-2)
plt.savefig(f"figs/validation/FigureEpiparamCompressed.pdf", bbox_inches='tight')

# ascertainment rates

In [ ]:
fig = plt.figure(figsize=(5.25, 3), dpi=300)

setup_large_plot()

traces = load_tagged_traces(results_base_dir, "scaling")
score = produce_score_sd(default_res, traces, derived_features_comb)

derived_features = derived_features_comb

width = 1
score = produce_score_sd(default_res, traces, derived_features_comb)
y_off = -np.linspace(-0.3, 0.3, len(traces)+1)
width = 1
for i, (trace, label) in enumerate(traces):
    add_trace_to_plot(trace, y_off[i], colors[i], label, 1, width)
add_trace_to_plot(default_res, y_off[-1], 'k', "Default", 1, width) 
plt.legend(shadow=True, fancybox=True, loc="upper left", bbox_to_anchor=(0.95, 1.01), fontsize=6, title=f'$\\bar{{\\sigma}}={score:.2f}\\%$', title_fontsize=8)
plt.title("Case Undercounting Correction", fontsize="10")
# plt.subplot(122)
# setup_large_plot(False, True, True)

# traces = load_tagged_traces(results_base_dir, "scaling", '-mean.txt')
# score = produce_score_sd(default_res, traces, derived_features_comb)

# derived_features = derived_features_comb

# width = 1
# score = produce_score_sd(default_res, traces, derived_features_comb)
# y_off = -np.linspace(-0.3, 0.3, len(traces)+1)
# width = 1
# traces.reverse()
# for i, (trace, label) in enumerate(traces):
#     add_trace_to_plot(trace, y_off[i], colors[i], label, 1, width)
# add_trace_to_plot(default_res_mean, y_off[-1], 'k', "Default", 1, width) 

# plt.legend(shadow=True, fancybox=True, loc="upper left", bbox_to_anchor=(0.95, 1.01), fontsize=6, title=f'$\\bar{{\\sigma}}={score:.2f}\\%$', title_fontsize=8)
plt.tight_layout(w_pad=0.25)
# plt.title("Case Undercounting Correction\nMean Effects", fontsize="10")
plt.savefig(f"figs/validation/FigureUndercounting.pdf", bbox_inches='tight')

# alpha noise scale results

In [ ]:
fig = plt.figure(figsize=(5.25, 3), dpi=300)
setup_large_plot()
traces = load_tagged_traces(results_base_dir, "alpha_noise_scale")
score = produce_score(default_res, traces)
y_off = -np.linspace(-0.3, 0.3, len(traces)+1)
width = 1

traces.sort(key=lambda t: float(t[1]))
for i, (trace, label) in enumerate(traces):
    add_trace_to_plot(trace, y_off[i], colors[i], label if label != "0.04" else "Default\n0.04", 1, width)

score = produce_score_sd(default_res, traces, derived_features_comb)
plt.legend(shadow=True, fancybox=True, loc="upper right", bbox_to_anchor=(1.35, 1.01), fontsize=8, title=f'$\\bar{{\\sigma}}={score:.2f}\\%$', title_fontsize=10)
plt.title("$\\sigma_i$ Prior Scale", fontsize=10)
plt.tight_layout()
plt.savefig(f"figs/validation/FigureAlphaNoises.pdf", bbox_inches='tight')